## Recipe Database!!

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import os
from PIL import Image
from io import BytesIO
import numpy as np

### 1. Read the CSV

In [2]:
df = pd.read_csv('recipes.csv')

In [3]:
df.head(2)

,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...


### 2. Investigate the data (any nans, missing values, unneeded attributes)

In [184]:
# number of rows with nans
nan_rows = df[df.isna().any(axis=1)]
print("Number of rows with NaN values:\n", len(nan_rows))

# Filter out rows with NaN values
df_clean = df.dropna()
print("\nLength of clean df without NaN values:\n", len(df_clean))


Number of rows with NaN values:
 459

Length of clean df without NaN values:
 631


### 3. add recipe_id attribute (recplaced the "Unnanmed:0")

In [185]:
df_clean = df_clean.rename(columns={'Unnamed: 0': 'recipe_id'})

In [186]:
def get_image_url(recipe_url):
    try:
        response = requests.get(recipe_url, timeout=10)  # Set a timeout for the request
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            img_tags = soup.find_all('img', class_=['recipe-image', 'img-placeholder', 'figure-media', 'gallery-image', 'image-placeholder', 'ingredient-image'])

            # img_tag = soup.find_all('img', class_='recipe-image')  # Adjust the selector based on AllRecipes structure
            return img_tags[0]['src'] if img_tags else None
        else:
            print(f"Failed to retrieve {recipe_url}: Status code {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Error fetching {recipe_url}: {e}")
        return None

def download_image(image_url, save_path):
    try:
        response = requests.get(image_url, timeout=10)
        img = Image.open(BytesIO(response.content))
        img.save(save_path)
    except requests.RequestException as e:
        print(f"Error downloading image {image_url}: {e}")

In [187]:
# Add a column to your DataFrame for image URLs
df_clean['image_url'] = df_clean['url'].apply(get_image_url)

In [190]:
# number of rows with nans
nan_rows = df_clean[df_clean.isna().any(axis=1)]
print("Number of rows with NaN values:\n", len(nan_rows))

# Filter out rows with NaN values
final_df = df_clean.dropna()
print("\nLength of clean df without NaN values:\n", len(final_df))


Number of rows with NaN values:
 631

Length of clean df without NaN values:
 0


In [ ]:
# Download and save images
for i, row in df_clean.iterrows():
    if row['img_src']:
        download_image(row['img_src'], f"images/{row['recipe_id']}.jpg")

# Save the updated DataFrame
# df.to_csv('your_dataset_with_images.csv', index=False)
df_clean.to_csv("recipes_clean.csv", index=False)

print("Images downloaded and dataset updated.")


### 4. Make the text of ingredients into a list

In [4]:
import re
import pandas as pd

def parse_ingredient(ingredient_text):
    # Regex pattern to detect numbers and measurements
    measurements = r'(\d+\s*\d*/*\d*|\d*/*\d+|\d*[½⅓⅔¼¾]|[½⅓⅔¼¾])\s*(small|large|cup|cups?|tablespoons?|tbsp|teaspoons?|tsp|pounds?|ounces?|inch|pinch|slices?|cloves?|quarts?|grams?|kg|ml|liters?)'
    
    # Split by commas and parse each ingredient part
    ingredients = ingredient_text.split(',')  # Split on commas

    
    return ingredients


In [6]:
recipe_df = pd.read_csv("recipes_clean.csv")

In [7]:
recipe_df["ingredients"].iloc[4]

"2 tablespoons lemon juice, 4 cups water, 4  Granny Smith apples - peeled, cored and sliced, 2 tablespoons butter, 1 cup brown sugar, 1 teaspoon ground cinnamon, 1 tablespoon cornstarch, 1 tablespoon water, 1 (17.25 ounce) package frozen puff pastry sheets, thawed, 1 cup confectioners' sugar, 1 tablespoon milk, 1 teaspoon vanilla extract"

In [8]:
recipe_df["cleaned_ingredients"] = recipe_df["ingredients"].apply(parse_ingredient)

In [17]:
recipe_df = recipe_df.drop(columns={"image_url"})

In [18]:
recipe_df.columns

Index(['recipe_id', 'recipe_name', 'prep_time', 'cook_time', 'total_time',
       'servings', 'yield', 'ingredients', 'directions', 'rating', 'url',
       'cuisine_path', 'nutrition', 'timing', 'img_src',
       'cleaned_ingredients'],
      dtype='object')

In [25]:
# Ensure the 'recipe_id' is of type string
recipe_df['recipe_id'] = recipe_df['recipe_id'].astype(str)

# Create the 'image_path' column using an f-string with the apply function
recipe_df['image_path'] = recipe_df['recipe_id'].apply(lambda x: f"./images/{x}.png")


In [26]:
recipe_df["image_path"]

0         ./images/1.png
1         ./images/3.png
2         ./images/4.png
3         ./images/7.png
4        ./images/10.png
             ...        
626    ./images/1083.png
627    ./images/1084.png
628    ./images/1085.png
629    ./images/1087.png
630    ./images/1088.png
Name: image_path, Length: 631, dtype: object

In [188]:
# import re
# from fractions import Fraction

# def parse_fraction(value):
#     """Convert a fraction string to a float."""
#     try:
#         return float(sum(Fraction(s) for s in value.split()))
#     except ValueError:
#         return value

# def parse_ingredient(ingredient_text):
#     # Regex pattern to detect numbers, fractions, and units
#     measurements = r'(\d+\s*\d*/*\d*|\d*/*\d+|\d*[\u00BC-\u00BE\u2150-\u215E]?\s*\d*[\u00BC-\u00BE\u2150-\u215E]?|[0-9]+\s*[½⅓⅔¼¾]|[½⅓⅔¼¾])\s*(small|large|cup|cups|tablespoons?|tbsp|teaspoons?|tsp|pounds?|ounces?|inch|pinch|slices?|cloves?|quarts?|grams?|kg|ml|liters?)\s*'
    
#     # Split by commas and parse each ingredient part
#     ingredients = re.split(r',\s*', ingredient_text)  # Split on commas, allowing for optional spaces
#     ingredient_dict = {}
    
#     for item in ingredients:
#         # Match the quantity and unit (if any)
#         match = re.match(measurements, item)
        
#         if match:
#             # Extract the quantity and unit
#             quantity = match.group(1)
#             unit = match.group(2)
            
#             # Extract the food item by removing the matched quantity and unit
#             food_item = item.replace(match.group(0), '').strip()
            
#             # Convert fractions if necessary
#             parsed_quantity = parse_fraction(quantity)
            
#             # Store in dictionary with both quantity and unit
#             ingredient_dict[food_item] = f"{parsed_quantity} {unit}"
#         else:
#             # If no match, store the ingredient without quantity
#             ingredient_dict[item.strip()] = None
    
#     return ingredient_dict

# # # Example usage
# # ingredient_text = '8 small Granny Smith apples, ½ cup unsalted butter, 3 tablespoons all-purpose flour, ½ cup white sugar, ½ cup packed brown sugar, ¼ cup water, 1 (9 inch) double-crust pie pastry, thawed'
# # parsed_ingredients = parse_ingredient(ingredient_text)

# # print(parsed_ingredients)


In [189]:
# import re
# import pandas as pd

# def parse_ingredient(ingredient_text):
#     # Regex pattern to detect numbers and measurements
#     measurements = r'(\d+\s*\d*/*\d*|\d*/*\d+|\d*[½⅓⅔¼¾]|[½⅓⅔¼¾])\s*(small|large|cup|cups?|tablespoons?|tbsp|teaspoons?|tsp|pounds?|ounces?|inch|pinch|slices?|cloves?|quarts?|grams?|kg|ml|liters?)'
    
#     # Split by commas and parse each ingredient part
#     ingredients = re.split(r',\s*', ingredient_text)  # Split on commas, allowing for optional spaces
#     ingredient_dict = {}
    
#     for item in ingredients:
#         # Match the quantity and unit (if any)
#         match = re.search(measurements, item)
        
#         if match:
#             # Extract the quantity and unit
#             quantity = match.group(1)
#             unit = match.group(2)
            
#             # Extract the food item by removing the matched quantity and unit
#             food_item = item.replace(match.group(0), '').strip()
            
#             # Store in dictionary with both quantity and unit
#             ingredient_dict[food_item] = f"{quantity} {unit}"
#         else:
#             # If no match, store the ingredient without quantity
#             ingredient_dict[item.strip()] = None
    
#     return ingredient_dict

# # Example DataFrame
# data = {
#     "ingredients": [
#         '4 ½ cups white sugar, 1 cup cornstarch, 2 teaspoons ground cinnamon, ¼ teaspoon ground nutmeg, 10 cups water, 2 teaspoons salt, 3 tablespoons lemon juice, 2 drops yellow food coloring, 6 pounds apples'
#     ]
# }

# # df_clean = pd.DataFrame(data)

# # # Apply the function to each row in the "ingredients" column
# # df_clean["cleaned_ingredients"] = df_clean["ingredients"].apply(parse_ingredient)

# # # Display the DataFrame
# # print(df_clean[["ingredients", "cleaned_ingredients"]])


### 4. Create a DB using sqlalchemy

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Create a base class for your models
Base = declarative_base()

# Define the Recipe class which represents the recipes table
class Recipe(Base):
    __tablename__ = 'recipes'
    
    recipe_id = Column(Integer, primary_key=True, autoincrement=True)
    recipe_name = Column(String)
    img_src = Column(String)
    image_path = Column(String)
    

# Create a SQLite database and a connection
engine = create_engine('sqlite:///recipes.db')

# Create the recipes table (if it doesn't exist)
Base.metadata.create_all(engine)


In [ ]:
# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Assuming your DataFrame (df) contains 'recipe_name', 'img_src', and 'recipe_id' columns
for _, row in df.iterrows():
    # Create a new Recipe object
    recipe = Recipe(
        recipe_name=row['recipe_name'],
        img_src=row['img_src'],
        image_path=f"images/{row['recipe_id']}.jpg"  # Assuming images are stored in the 'images/' folder
    )
    
    # Add the recipe to the session
    session.add(recipe)

# Commit all changes to the database
session.commit()

# Close the session
session.close()


In [ ]:
# Create a new session for querying
session = Session()

# Query all recipes from the database
recipes = session.query(Recipe).all()

# Print the recipes and their image paths
for recipe in recipes:
    print(f"Recipe: {recipe.recipe_name}, Image Path: {recipe.image_path}")

session.close()


In [ ]:
import os

# Directory to save images
os.makedirs('images', exist_ok=True)

# Assuming you have image URLs in your DataFrame and want to download them
for i, row in df.iterrows():
    image_url = row['img_src']
    image_path = f"images/{row['recipe_id']}.jpg"
    
    try:
        # Download and save the image
        response = requests.get(image_url, timeout=10)
        img = Image.open(BytesIO(response.content))
        img.save(image_path)
    except Exception as e:
        print(f"Error downloading image for recipe {row['recipe_name']}: {e}")


In [ ]:
from sqlalchemy import LargeBinary

class Recipe(Base):
    __tablename__ = 'recipes'
    
    recipe_id = Column(Integer, primary_key=True, autoincrement=True)
    recipe_name = Column(String)
    img_src = Column(String)
    image_data = Column(LargeBinary)  # Store image data as BLOB


In [29]:
from sqlalchemy import create_engine

# Create an SQLite engine
engine = create_engine('sqlite:///recipe_database.db')

# Save the DataFrame to a SQL table named "recipes"
recipe_df.to_sql('recipe_database', con=engine, if_exists='replace', index=False)


InterfaceError: (sqlite3.InterfaceError) Error binding parameter 15 - probably unsupported type.
[SQL: INSERT INTO recipe_database (recipe_id, recipe_name, prep_time, cook_time, total_time, servings, yield, ingredients, directions, rating, url, cuisine_path, nutrition, timing, img_src, cleaned_ingredients, image_path) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [('1', 'Apple Pie by Grandma Ople', '30 mins', '1 hrs', '1 hrs 30 mins', 8, '1 9-inch pie', '8 small Granny Smith apples, or as needed, ½ cup unsalted butter, 3 tablespoons all-purpose flour, ½ cup white sugar, ½ cup packed brown sugar, ¼ cup water, 1 (9 inch) double-crust pie pastry, thawed', "Peel and core apples, then thinly slice. Set aside.\nPreheat the oven to 425 degrees F (220 degrees C).\nMelt butter in a saucepan over medium heat.  ... (1389 characters truncated) ... duce the temperature to 350 degrees F (175 degrees C) and continue baking until apples are soft, 35 to 45 minutes.\n\n\n\n\n\